In [ ]:
!pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 68 kB 3.0 MB/s 
  Using cached pybind11-2.10.1-py3-none-any.whl (216 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3165480 sha256=621db7815baebd4f7448d8dcbdb7f4b4ed8910642e6fd02014dea0cc7a5dc156
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [ ]:
import logging
import pandas as pd
import nltk
import re
from bs4 import BeautifulSoup
import numpy as np
import gensim
from gensim.models.fasttext import FastText
from nltk.tokenize import sent_tokenize, word_tokenize
import csv
import warnings
warnings.filterwarnings ('ignore')

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#df = pd.read_csv('GitterCom.csv', header=0, encoding='UTF8')
#df['message'].to_csv('alice.txt', sep="\n", index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
sample = open("/content/gdrive/My Drive/Gitter_Dataset/alice.txt", "r",encoding="utf8")
s = sample.read()
f = s.replace("\n", " ")

data = []
for i in sent_tokenize(f):
	temp = []

	# tokenize the sentence into words
	for j in word_tokenize(i):
		temp.append(j.lower())

	data.append(temp)

model = FastText(data, min_count = 1, size = 100, window = 5, sg = 1)

In [ ]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS =nltk.corpus.stopwords.words('english')

def clean_text(text):
    if type(text)==str:

        text = BeautifulSoup(text, "lxml").text # HTML decoding
        text = text.lower() # lowercase text
        text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
        text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
        text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
        return text
    return ""
def wordprint(words,model1):
    all_words, mean = set(), []
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in model1.wv.vocab:
            a=np.array(model1[word])
            mean.append(a)
    if not mean:
        # FIXME: remove these examples in pre-processing
        return np.zeros(100,)
    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

In [ ]:
def  word_averaging_listn(wv, text_list):
    return np.vstack([wordprint(post,wv) for post in text_list ])

def w2v_tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text, language='english'):
        for word in nltk.word_tokenize(sent, language='english'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens

In [ ]:
import numpy as np

fname='/content/gdrive/My Drive/Gitter_Dataset/GitterCom'+'.csv'
df = pd.read_csv(fname,encoding='UTF8')
df.drop([581, 2102, 4858, 6417], axis=0, inplace=True)

non_string = []
for i, row in df.iterrows():
    res = ((type(row['message']) != str) or (type(row['Purpose']) != str) or (type(row['Category']) != str) or (type(row['Subcategory']) != str))
    if res:
        non_string.append(i)

df.drop(non_string, axis=0, inplace=True)
len(df)

9959

In [ ]:
df.drop(['Channel','messageId','time','user'],axis=1,inplace=True)
df['Category']=df['Category'].str.lower()
df['Purpose']=df['Purpose'].str.lower()
df['Subcategory']=df['Subcategory'].str.lower()
df['message'] = df['message'].apply(clean_text)
print(df.head(10))

                                             message    Purpose  \
0  hi team recently upgraded cucumberjvm version ...  team wide   
1  exception thread main javalangnosuchmethoderro...  team wide   
2  amit007 looks like inconsistent cucumber jar v...  team wide   
3                        github trying replace irc p  team wide   
4  aslakhellesoy thanks seems like using older ve...  team wide   
5  hi one question friends starting project studi...  team wide   
6  danon9111 integrate cucumber another testing f...  team wide   
7  aslakhellesoy reading cucumber book found code...  team wide   
8                 sidkiyassine call methods directly  team wide   
9  intergrated cucumber+appium https githubcom pr...  team wide   

        Category                          Subcategory  
0        dev-ops  development operation notifications  
1        dev-ops  development operation notifications  
2  communication         communication with teammates  
3  communication         communication

In [ ]:
comtdata=df
test_tokenized = comtdata.apply(lambda r: w2v_tokenize_text(r['message']), axis=1).values
X_comtdata_average1 = word_averaging_listn(model, test_tokenized)
fname='7'+'.csv'   #fasttext
np.savetxt(fname,X_comtdata_average1, delimiter=',', fmt='%f')